In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('./deeplearning.mplstyle')

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

np.set_printoptions(precision=2)

Preprocessing of the data
First we drop the unused data, or NA values
Then we convert the genres into numerical values so it can be used by the machine learning algorithm



In [3]:
#load data, drop unnecessary columns, and drop all NA values
df = pd.read_csv('music_genre.csv')
df = df.drop(['instance_id', 'artist_name', 'track_name', 'key', 'mode','obtained_date','tempo'], axis=1) 
df = df.dropna()
df = df.sample(frac=1)

In [4]:
print(df.dtypes)

popularity          float64
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
valence             float64
music_genre          object
dtype: object


In [5]:



# Filter the DataFrame to include only the selected genres
encoder = LabelEncoder()
df['music_genre'] = encoder.fit_transform(df['music_genre'])
# df['mode'] = encoder.fit_transform(df['mode'])
# df['key'] = encoder.fit_transform(df['key'])
# df['tempo'] = pd.to_numeric(df['tempo'].str.replace('.', ''), errors='coerce')
# df['tempo'] = df['tempo'].fillna(0)


In [6]:
X = df.drop('music_genre', axis=1)
Y = df['music_genre']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = StandardScaler()
size = X_train.shape[1]


In [7]:
# UNQ_C3
# GRADED CELL: model
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

tf.random.set_seed(1234)
model = Sequential(
    [
        ### START CODE HERE ### 
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax'),
        
        
        ### END CODE HERE ### 

    ], name="Complex"
)
model.compile(
    ### START CODE HERE ### 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['accuracy']

    ### END CODE HERE ### 
)

In [10]:
train_2 = model.fit(X_train , Y_train , validation_data=(X_test,Y_test),epochs=200, verbose=1)

Epoch 1/200
1250/1250 [==============================] - 3s 2ms/step - loss: 2.2002 - accuracy: 0.1379 - val_loss: 2.1903 - val_accuracy: 0.1416
Epoch 2/200
1250/1250 [==============================] - 3s 2ms/step - loss: 2.3957 - accuracy: 0.1404 - val_loss: 2.1914 - val_accuracy: 0.1401
Epoch 3/200
1250/1250 [==============================] - 3s 2ms/step - loss: 2.5590 - accuracy: 0.1392 - val_loss: 2.1884 - val_accuracy: 0.1429
Epoch 4/200
1250/1250 [==============================] - 3s 2ms/step - loss: 2.3139 - accuracy: 0.1431 - val_loss: 2.1892 - val_accuracy: 0.1446
Epoch 5/200
1250/1250 [==============================] - 3s 2ms/step - loss: 2.2422 - accuracy: 0.1412 - val_loss: 2.1886 - val_accuracy: 0.1466
Epoch 6/200
1250/1250 [==============================] - 3s 2ms/step - loss: 3.0372 - accuracy: 0.1389 - val_loss: 2.1886 - val_accuracy: 0.1424
Epoch 7/200
1250/1250 [==============================] - 3s 2ms/step - loss: 5.1635 - accuracy: 0.1447 - val_loss: 2.1881 - val_ac